In [1]:
from pathlib import Path
import os
import sys

# Setup for importing custom modules.
sys.path.insert(0, str(Path(os.getcwd()).parent.parent))

In [2]:
from typing import Any

import numpy as np

from matplotlib import animation
from matplotlib import pyplot as plt

from scipy.integrate import solve_ivp
from scipy.fftpack import fft2, ifft2

from IPython.display import HTML

%matplotlib notebook

$$
\begin{align*}
\frac {\partial}{\partial t} \left(\begin{align*} u \\ v \end{align*}\right)
    &=
    \left( \begin{align*} &\lambda (A) &-\omega (A) \\ &\omega (A) &\lambda (A) \end{align*} \right)
    \left( \begin{align*} u \\ v \end{align*} \right)
    + D \nabla^2 \left( \begin{align*} u \\ v \end{align*} \right) \\
    &=
    \left( \begin{align*} &1 - A^2 &\beta A^2 \\ &-\beta A^2 & 1 - A^2 \end{align*} \right)
    \left( \begin{align*} u \\ v \end{align*} \right)
    + D \nabla^2 \left( \begin{align*} u \\ v \end{align*} \right)
\end{align*}
$$
with $A^2 = u^2 + v^2$.
Furthermore this results in the following system
$$
U_t = \lambda(A)U - \omega(A)V + D_1 \nabla^2 U \\
V_t = \omega(A)U + \lambda(A)V + D_2 \nabla^2 V
$$
plugging in what we have for $\lambda(A) = 1 - A^2$ and $\omega(A) = -\beta A^2$, gives us
$$
\begin{align*}
U_t &= (1 - A^2) U + \beta A^2 V + D_1 \nabla^2 U \\
V_t &= - \beta A^2 U + (1 - A^2) V + D_2 \nabla^2 V.
\end{align*}
$$
We can also advance things further by plugging in our values of $\beta$ and $A^2$
$$
\begin{align*}
U_t &= U - U^3 + V^2 U + \beta U^2 V + V^3 + D_1 \nabla^2 U \\
V_t &= - \beta U^3 -\beta V^2 U + V - U^2 v + V^3 + D_2 \nabla^2 V.
\end{align*}
$$
Finally the second gradient operator is given by $\nabla^2 = \partial_x^2 + \partial_y^2$, therefore we have
$$
\begin{align*}
U_t &= U - U^3 + V^2 U + \beta U^2 V + V^3 + D_1 ( \partial_x^2 U + \partial_y^2 U ) \\
V_t &= - \beta U^3 -\beta V^2 U + V - U^2 v + V^3 + D_2 ( \partial_x^2 V + \partial_y^2 V ).
\end{align*}
$$

In [3]:
def compute_a_matrix(u: np.array, v: np.array):
    """Nadda"""
    return u*u + v*v


def non_linear_lambda(a_matrix: np.array) -> np.array:
    """Nadda"""
    return 1 - a_matrix
    

def non_linear_omega(a_matrix: np.array, beta: float = 1) -> np.array:
    """Nadda"""
    return -beta * a_matrix


def fourier_laplacian(mat: np.array, n: int, kx, ky) -> np.array:
    """"""
    # vector = mat.reshape((n + 1) ** 2)

    derivative_terms = kx**2 + ky**2
    laplacian = -derivative_terms * fft2(mat)
    
    return ifft2(laplacian)


def setup_chebyshev(n: int):
    """
    Determine the x vlaues to evaluate at

    :param n: the number of points to evaluate at

    :returns: the derivative matrix and x
    """
    if n==0:
        D = 0
        x = 1
    else:
        indices = np.arange(0,n+1)
        x = np.cos(np.pi*indices/n).reshape(n+1,1)
        c = (
                (-1)**indices * np.hstack(([2.], np.ones(n-1), [2.]))
            ).reshape(n+1,1)
        X = np.tile(x, (1,n+1) )
        dX = X - X.T
        D = np.dot(c , 1./c.T ) / (dX + np.eye(n+1))
        D = D - np.diag( np.sum(D.T, axis=0) )

    return D, x.reshape(n+1)


def cheb_laplacian(mat: np.array, D: np.array, n: int) -> np.array:
    """"""
    vector = mat.reshape((n + 1) ** 2)

    D[n, :] = 0
    D[0, :] = 0
    D2 = np.dot(D, D)/(10**2)

    I = np.eye(len(D2))
    L = np.kron(I, D2) + np.kron(D2, I)  # 2D Laplacian

    laplacian = np.dot(L, vector).reshape((n + 1, n + 1))

    return laplacian


def laplacian(mat: np.array, method: str = "fourier", laplacian_params = dict[str, Any]) -> np.array:
    """
    Compute the laplacian of the provided Matrix

    :param mat: matrix to compute the laplacian of

    :returns: the laplacian
    """
    if method == "fourier":
        lap = fourier_laplacian(mat, **laplacian_params)
    elif method == "cheb":
        lap = cheb_laplacian(mat, **laplacian_params)
    else:
        raise ValueError(f"Method {method} has not been implimented yet")

    return lap


def reaction_diffusion_rhs(
    t: float,
    y: np.array,
    n: int,
    laplacian_method: str,
    laplacian_params: dict[str, Any],
    D_1: float = 0.1,
    D_2: float = 0.1,
    beta: float = 1,
) -> np.array:
    """
    Solve the reaction diffusion system.

    :param y: the full array representing U and V.
    :param t: the tspan
    :param n: the dimension of the uv
    :param laplacian_method: str describing method of taking derivatives
    :param laplacian_params: help setup the various laplacian methods
    :param D_1: the coefficient of the laplacian for U
    :param D_2: the coefficient of the laplacian for V
    :param beta: scalar in the nonlinear omega term

    :returns: the dervaive of u and v with respect to t stacked
    """
    u, v = np.split(y,[(n+1)**2])
    U = u.reshape((n+1,n+1))
    V = v.reshape((n+1,n+1))
    a_matrix = compute_a_matrix(U, V)

    lambda_A = non_linear_lambda(a_matrix)
    omega_A = non_linear_omega(a_matrix, beta=beta)

    laplacian_U = laplacian(U, method=laplacian_method, laplacian_params=laplacian_params)
    laplacian_V = laplacian(V, method=laplacian_method, laplacian_params=laplacian_params)

    U_t = lambda_A*U - omega_A*V + D_1*laplacian_U
    V_t = omega_A*U + lambda_A*V + D_2*laplacian_V

    u_t = U_t.reshape((n+1)**2)
    v_t = V_t.reshape((n+1)**2)

    y_t = np.hstack((u_t, v_t))
    
    return y_t


In [4]:
def initial_condition(X: np.array, Y: np.array, m: int = 1):
    """"""
    U0 = np.tanh(np.sqrt( X**2 + Y**2)) * np.cos( m * np.angle(X + Y*1j) - np.sqrt(X**2 + Y**2))
    V0 = np.tanh(np.sqrt( X**2 + Y**2)) * np.sin( m * np.angle(X + Y*1j) - np.sqrt(X**2 + Y**2))
    return U0, V0


def hw_problem_1():
    """Define the setup for the fourier solve with periodic boundary conditions"""
    L = 10
    n = 64
    m = 1
    tspan = np.arange(0, 20.5, .5)

    x_full = np.linspace(-L, L, n + 1)
    y_full = np.linspace(-L, L, n + 1)

    x_trunc = x_full[:n]
    y_trunc = y_full[:n]

    X, Y = np.meshgrid(x_trunc, y_trunc)

    scale_factor = 2 * np.pi / (L - (-L))
    kx = scale_factor * np.concatenate((np.arange(0, n/2), np.arange(-n/2, 0)))
    ky = scale_factor * np.concatenate((np.arange(0, n/2), np.arange(-n/2, 0)))

    # avoid divide by zero with floating point precision error
    kx[0] = 1e-6
    ky[0] = 1e-6
    kx, ky = np.meshgrid(kx, ky)
    
    U0, V0 = initial_condition(X, Y, m=m)
    y0 = np.hstack(
        (U0.reshape(n**2), V0.reshape(n**2))
    )

    fourier_params = {
        "n": 63,
        "kx": kx,
        "ky": ky,
    }
    laplacian_method = "fourier"
    D_1 = 0.1
    D_2 = 0.1
    beta = 1

    sol = solve_ivp(
        reaction_diffusion_rhs,
        t_span=(tspan[0], tspan[-1]),
        y0=y0,
        t_eval=tspan,
        args=(n-1, laplacian_method, fourier_params, D_1, D_2, beta)
    )
    return sol


def hw_problem_2():
    """Define the setup for the chebyshev solve with no flux boundary conditions"""
    n = 30
    m = 1
    tspan = np.arange(0, 4.5, .5)
    D, x = setup_chebyshev(30)
    cheb_params = {
        "n": n,
        "D": D,
    }
    X, Y = np.meshgrid(10*x, 10*x)
    U0, V0 = initial_condition(X, Y, m=m)

    y0 = np.hstack(
        (U0.reshape((n+1)**2), V0.reshape((n+1)**2))
    )

    laplacian_method = "cheb"
    D_1 = 0.1
    D_2 = 0.1
    beta = 1

    sol = solve_ivp(
        reaction_diffusion_rhs,
        t_span=(tspan[0], tspan[-1]),
        y0=y0,
        t_eval=tspan,
        args=(n, laplacian_method, cheb_params, D_1, D_2, beta)
    )
    return sol

In [5]:
sol = hw_problem_1()
sol.y.shape
U, V = np.split(sol.y, [(64)**2])

/Users/hunterlybbert/.pyenv/versions/uw-dev-env/lib/python3.11/site-packages/scipy/integrate/_ivp/base.py:23: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(fun(t, y), dtype=dtype)


In [6]:
animation.writer = animation.writers['ffmpeg']

plt.ioff()
fig = plt.figure()
ax = fig.add_subplot(111)

# write the update function, specifically including the ax.clear() function this was important.
def update(i):
    ax.clear()
    ax.pcolor(np.real(U[:,i].reshape((64, 64))), cmap='bwr')
    ax.set_title("Evolution of Vorticity Stream Function")
    return ax

ani = animation.FuncAnimation(fig, update, frames=range(len(sol.t)), interval=125)

HTML(ani.to_html5_video())

In [7]:
sol = hw_problem_2()
U, V = np.split(sol.y, [(30+1)**2])

In [8]:
animation.writer = animation.writers['ffmpeg']

plt.ioff()
fig = plt.figure()
ax = fig.add_subplot(111)

# write the update function, specifically including the ax.clear() function this was important.
def update(i):
    ax.clear()
    ax.pcolor(U[:,i].reshape((30 + 1, 30 + 1)), cmap='bwr')
    ax.set_title("Evolution of Vorticity Stream Function")
    return ax

ani = animation.FuncAnimation(fig, update, frames=range(9), interval=125)

HTML(ani.to_html5_video())